In [70]:
llm_file = "../../evaluation/evaluation_sanitized-mbpp.json"

In [71]:
import json

file = open(llm_file, 'r')
tasks = json.load(file)

In [72]:
import ast

task = tasks[0]
task["task_id"]

2

In [73]:
print("\n".join(task["test_list"]))

assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4),(5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13),(17, 15, 14, 13))) == set((13, 14))


In [74]:
test_ast = ast.parse(task["test_list"][0])

In [ ]:
Set()